<a href="https://colab.research.google.com/github/Anti-puff-dev/gpt-python/blob/main/nlp_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MODELS

In [ ]:
!kill -9 -1

In [ ]:
!pip install protobuf==3.19.0

In [ ]:
from typing import List, Tuple, Dict, Iterable, Optional
import re
import math
import itertools
import json
import math
import os
from collections import Counter
import pickle
import time

In [ ]:
class Vocab:
    def __init__(self, word: str, word_partial: str, count: int):
        self.word = word
        self.word_partial = word_partial
        self.count = count

In [ ]:
class Token:
    def __init__(self, index: int, count: int, position: List[int], weight: float, nonce: bool):
        self.index = index
        self.count = count
        self.position = position
        self.weight = weight
        self.nonce = nonce

In [ ]:
class Tensor:
    def __init__(self, category_id, word_id, weight):
        self.category_id = category_id
        self.word_id = word_id
        self.weight = weight

In [ ]:
class GenVec:
    def __init__(self, index: int, count: int, weight: float):
        self.index = index
        self.count = count
        self.weight = weight

In [ ]:
class Similarity:
    def __init__(self, c1, c2, value):
        self.c1 = c1
        self.c2 = c2
        self.value = value

In [ ]:
class Embedding:
    def __init__(self, values = None):
        self.values = values if values is not None else []

class WordVector:
    def __init__(self, embeddings = None):
        self.embeddings = embeddings if embeddings is not None else []

CLASSES

In [ ]:
class Tokenize:
    def __init__(self, word_pooling: float = 1.0):
        self.word_pooling = word_pooling

    @staticmethod
    def instance(word_pooling: float = 1.0) -> 'Tokenize':
        return Tokenize(word_pooling)

    def add_endings(self, lists: List[List[str]]) -> List[List[str]]:
        for i in range(len(lists)):
            for j in range(len(lists[i])):
                lists[i][j] += " <end>"
        return lists

    def vocab(self, lists: List[List[str]]) -> List[Vocab]:
        lists = self.add_endings(lists)
        vocab_dict = {}

        for lst in lists:
            for sentence in lst:
                words = self.pontuation_v(sentence).lower().split()
                for word in words:
                    if word in vocab_dict:
                        vocab_dict[word] += 1
                    else:
                        vocab_dict[word] = 1

        return sorted(
            [
                Vocab(word, self.word_pooling_func(word, self.word_pooling), count)
                for word, count in vocab_dict.items()
            ],
            key=lambda v: v.word,
        )



    def embedding(self, category_id: int, text: str, vocab: List[Vocab]) -> List[int]:
        embed = [category_id]
        words = self.pontuation(text + " <end>").lower().split()

        for word in words:
            index = next((i for i, v in enumerate(vocab) if v.word == word), -1)
            if index != -1:
                embed.append(index)

        return embed

    def apply(self, text: str, vocab: List[Vocab]) -> Tuple[List[Token], int]:
        tokens = []
        lst = self.pontuation(text + " <end>").split()
        result = {word: len(list(group)) for word, group in itertools.groupby(sorted(lst))}

        for word, count in result.items():
            word_index = next((i for i, v in enumerate(vocab) if v.word_partial == self.word_pooling_func(word, self.word_pooling)), -1)
            if word_index != -1:
                ids = self.get_ids(lst, word)
                is_nonce = self.is_nonce(word)
                tokens.append(Token(word_index, count, list(ids), 0 if is_nonce else 1, is_nonce))

        return tokens, len(lst)

    def pontuation(self, text: str) -> str:
        text = text.lower()
        for char in [",", ".", "*", ":", "'"]:
            text = text.replace(char, f" {char} ")
        for char in ["?", "!"]:
            text = text.replace(char, "")
        return " ".join(text.split())

    def pontuation_v(self, text: str) -> str:
        text = text.lower()
        for char in [",", ".", "*", ":", "'"]:
            text = text.replace(char, f" {char} ")
        for char in ["?", "!"]:
            text = text.replace(char, " ")
        return " ".join(text.split())

    @staticmethod
    def word_pooling_func(word: str, rate: float) -> str:
        length = len(word)
        if length <= 4:
            return word
        pad = math.ceil(length * rate)
        return word[:pad]

    @staticmethod
    def get_ids(lst: List[str], word: str) -> Iterable[int]:
        return [i for i, w in enumerate(lst) if w == word]

    @staticmethod
    def is_nonce(word: str) -> bool:
        nonce_words = {
            "das", "da", "de", "do", "dos", "na", "nas", "no", "nos", "em", "o", "a", "as", "os",
            "e", "com", "que", "ao", "aos", " , ", " . ", " : ", " ? ", " ! ", " ' ", " * ", ",",
            ".", ":", "?", "!", "'", "*", "<end>"
        }
        return word in nonce_words

    def print_vocab(self, vocab: List[Vocab]):
        for item in vocab:
            print(f"{item.word} {item.word_partial} {item.count}")

    def print_tokens(self, tokens: List[Token], vocab: List[Vocab]):
        for item in tokens:
            print(f"{item.index} {item.count} {vocab[item.index].word}")

    def print_ids(self, ids: Iterable[int]):
        for item in ids:
            print(item)

In [ ]:
class CategorySmall:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name

class Category:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name
        self.tokens: List[Token] = []
        self._len = 0

    def add_tokens(self, data: Tuple[List[Token], int]):
        tokens, length = data

        for token in tokens:
            index = next((i for i, t in enumerate(self.tokens) if t.index == token.index), -1)

            for i in range(len(token.position)):
                token.position[i] += self._len

            if index == -1:
                self.tokens.append(Token(token.index, token.count, token.position, token.weight, token.nonce))
            else:
                self.tokens[index].count += token.count
                self.tokens[index].position.extend(token.position)

        self._len += length


    def test(self, word: str, next_word: str, vocab: List['Vocab']):
        index0 = next((i for i, v in enumerate(vocab) if v.word == word), -1)
        index1 = next((i for i, v in enumerate(vocab) if v.word == next_word), -1)

        t1 = next((t for t in self.tokens if t.index == index0), None)
        t2 = next((t for t in self.tokens if t.index == index1), None)

        if index0 > -1 and index1 > -1 and t1 is not None and t2 is not None:
            near_position = self.alpha_minor_distance_position(t1.position, t2.position)
            print(f"{word} {next_word}| {near_position}")

        print(f"{word} {next_word}| {1000}")

    def alpha_minor_distance_position(self, position: List[int], ids: List[int]) -> int:
        dist = 10000
        rid = 0

        for pos in position:
            for id in ids:
                dst = abs(pos - id)
                if dst < dist:
                    dist = dst
                    rid = id

        return dist

    @staticmethod
    def multiply(vector: List[float], matrix: List[List[float]]) -> List[float]:
        return [sum(vector[j] * matrix[j][i] for j in range(len(vector))) for i in range(len(matrix[0]))]

    @staticmethod
    def dot(vector1: List[float], vector2: List[float]) -> float:
        return sum(v1 * v2 for v1, v2 in zip(vector1, vector2))

    @staticmethod
    def softmax(vector: List[float]) -> List[float]:
        exp_values = [math.exp(v) for v in vector]
        sum_exp = sum(exp_values)
        return [v / sum_exp for v in exp_values]

    @staticmethod
    def multiply_scalar(scalar: float, vector: List[float]) -> List[float]:
        return [scalar * v for v in vector]

    @staticmethod
    def sum(vectors: List[List[float]]) -> List[float]:
        return [sum(vector[i] for vector in vectors) for i in range(len(vectors[0]))]

In [ ]:
class Model:
    def __init__(self, tensors: List[Tensor] = None, vocab: List[Vocab] = None, categories: List[CategorySmall] = None, vectors: WordVector = None):
        self.tensors = tensors or []
        self.vocab = vocab or []
        self.categories =  categories or []
        self.vectors = vectors or WordVector()

In [ ]:
!protoc --python_out=. modelproto.proto

In [ ]:
import sys
sys.path.append("/content")

In [ ]:
import modelproto_pb2

In [ ]:
class ProtobufConverter:
    @staticmethod
    def to_proto(model: Model) -> modelproto_pb2.ModelProto:
        proto_model = modelproto_pb2.ModelProto()

        # Converter tensors
        for tensor in model.tensors:
            proto_tensor = proto_model.tensors.add()
            proto_tensor.category_id = tensor.category_id
            proto_tensor.word_id = tensor.word_id
            proto_tensor.weight = tensor.weight

        # Converter vocab
        for vocab in model.vocab:
            proto_vocab = proto_model.vocab.add()
            proto_vocab.word = vocab.word
            proto_vocab.word_partial = vocab.word_partial

        # Converter categories
        for category in model.categories:
            proto_category = proto_model.categories.add()
            if isinstance(category, dict):
                proto_category.id = category["id"]
                proto_category.name = category["name"]
            else:  # Handle CategorySmall objects
                proto_category.id = category.id
                proto_category.name = category.name

        # Converter vectors
        #proto_model.vectors = modelproto_pb2.WordVectorProto()
        if isinstance(model.vectors, dict):  # Handle dictionaries
            for emb in model.vectors["embeddings"]:
                proto_emb = proto_model.vectors.embeddings.add()
                proto_emb.values.extend(emb["values"])
        else:  # Handle WordVector objects
            for emb in model.vectors.embeddings:
                proto_emb = proto_model.vectors.embeddings.add()
                proto_emb.values.extend(emb.values)

        return proto_model

    @staticmethod
    def from_proto(proto_model: modelproto_pb2.ModelProto) -> Model:
        model = Model()

        # Converter tensors
        for proto_tensor in proto_model.tensors:
            tensor = Tensor(
                category_id=proto_tensor.category_id,
                word_id=proto_tensor.word_id,
                weight=proto_tensor.weight
            )
            model.tensors.append(tensor)

        # Converter vocab
        for proto_vocab in proto_model.vocab:
            vocab = Vocab(
                word=proto_vocab.word,
                word_partial=proto_vocab.word_partial,
                count=proto_vocab.count
            )
            model.vocab.append(vocab)

        # Converter categories
        for proto_category in proto_model.categories:
            category = CategorySmall(
                id=proto_category.id,
                name=proto_category.name
            )
            model.categories.append(category)

        # Converter vectors
        model.vectors = WordVector()
        for proto_emb in proto_model.vectors.embeddings:
            emb = Embedding(values=list(proto_emb.values))
            model.vectors.embeddings.append(emb)

        return model

    @staticmethod
    def save_to_file(model: Model, file_path: str):
        proto_model = ProtobufConverter.to_proto(model)
        with open(file_path, "wb") as f:
            f.write(proto_model.SerializeToString())

    @staticmethod
    def load_from_file(file_path: str) -> Model:
        with open(file_path, "rb") as f:
            proto_model = modelproto_pb2.ModelProto()
            proto_model.ParseFromString(f.read())
            return ProtobufConverter.from_proto(proto_model)

In [ ]:

class Classify:
    def __init__(self, model_path=None, vocab=None, categories=None, tensors=None, vectors=None):
        self.word_pooling  = 1.0
        self.dropout = 0.0001
        self.classes_number = 3
        self.non_category_threshold = 0.05
        self.distance_temperature = 1.0

        self.categories = categories or []
        self.tensors = tensors or []
        if vocab != None: self.vocab = vocab or []
        self.vectors = vectors or []
        self.model_path = model_path or ""

    @staticmethod
    def instance(model_path=None, vocab=None, categories=None, tensors=None, vectors=None):
        return Classify(model_path, vocab, categories, tensors, vectors)

    def word_pooling(self, word_pooling):
        self.word_pooling = word_pooling
        return self

    def mode(self, mode):
        self.mode = mode
        return self

    def vocab(self, vocab: List[Vocab]):
        self.vocab = vocab
        return self

    def distance_temperature(self, distance_temperature):
        self.distance_temperature = distance_temperature
        return self

    def model(self, model_path, read=False):
        self.model_path = model_path
        if read and self._file_exists(model_path):
            self.tensors, self.vocab, _categories, self.vectors = self.load_model(model_path)
            self.categories = self._fit_categories_from_small(_categories)
        return self


    def _file_exists(self, file_path: str) -> bool:
        return os.path.isfile(file_path)

    def dropout(self, dropout):
        self.dropout = dropout
        return self

    def classes_number(self, classes_number):
        self.classes_number = classes_number
        return self

    def non_category_threshold(self, nct):
        self.non_category_threshold = nct
        return self

    def add_category(self, name, lists):
      if isinstance(lists[0], list):
        for lst in lists:
            for text in lst:
                self._add_category(name, text)
      else:
        for text in lists:
            self._add_category(name, text)
      return self

    def _add_category(self, name, text):
        index = next((i for i, c in enumerate(self.categories) if c.name == name), -1)
        if index == -1:
            category_id = len(self.categories) + 1
            category = Category(id=category_id, name=name)
            category.add_tokens(Tokenize.instance(self.word_pooling).apply(text, self.vocab))
            self.categories.append(category)
            ids = Tokenize(self.word_pooling).embedding(category_id, text, self.vocab)
            self.vectors.append(ids)
        else:
            self.categories[index].add_tokens(Tokenize(self.word_pooling).apply(text, self.vocab))
            ids = Tokenize(self.word_pooling).embedding(self.categories[index].id, text, self.vocab)
            self.vectors.append(ids)
        return self

    def auto_categorize(self, lists):
        flat_list = []
        if isinstance(lists[0], list):
          for lst in lists:
              flat_list.extend(lst)
        else:
          flat_list = lists

        sims = []
        for i in range(len(flat_list) - 1):
            for j in range(i + 1, len(flat_list)):
                similarity = self._similarity(flat_list[i], flat_list[j])
                sims.append(Similarity(c1=i, c2=j, value=similarity))

        classes = []
        for i in range(len(flat_list)):
            tmp = [s for s in sims if s.c1 == i or s.c2 == i]
            tmp.sort(key=lambda x: x.value, reverse=True)
            classes.append(tmp)

        list_classes = []
        for i in range(self.classes_number):
            fx = math.ceil((len(classes[0]) - 1) / (self.classes_number - 1) * i)
            cx = classes[0][int(fx)].c2  # fx can be float, cast to int
            list_classes.append([cx])

        m = 0
        c = 1
        limit = len(flat_list) - self.classes_number

        while m < limit:
            for i in range(self.classes_number):
                j = list_classes[i][0]
                cy = classes[j][c].c2 if classes[j][c].c1 in list_classes[i] else classes[j][c].c2

                if not any(cy in sublist for sublist in list_classes):
                    list_classes[i].append(cy)
                    m += 1

                if m >= limit:
                  break
            c += 1

        class_index = 0
        for cla in list_classes:
            for x in cla:
                self._add_category(f"cat{class_index}", flat_list[x])
            class_index += 1
        return self

    def train(self):
        print("Training Start")
        self._race()
        self._fit()

        if not self.model_path:
            self.model_path = f"conquerer-{self.mode.lower()}-{self.word_pooling}-model.bin"
        self.save_model(self.model_path)
        print("Training Finished")

    def _race(self):
        percent = 0
        all_combinations = sum(range(1, len(self.categories)))

        for i in range(len(self.categories) - 1):
            for j in range(i + 1, len(self.categories)):
                c1, c2 = self._conquerer(self.categories[i], self.categories[j])
                self.categories[i] = c1
                self.categories[j] = c2

                percent += 1
                p = 100 * (percent + 1) / all_combinations

                print(f"Training {c1.name} -> {c2.name} {percent + 1}/{all_combinations} {math.ceil(p)}% {'#' * int(math.ceil(p / 4))}", end='\r') # \r to overwrite the line
        print() # add a newline after the loop

    def _fit(self):
        self.tensors.clear()
        for category in self.categories:
            for token in category.tokens:
               #print(str(category.id) + " " + str(token.index) + " " + str(token.weight))
                self.tensors.append(Tensor(category_id=category.id, word_id=token.index, weight=token.weight))

    def _conquerer(self, c1, c2):
        tokens = self._intersect(c1.tokens, c2.tokens)

        for token in tokens:
            index1 = next((i for i, t in enumerate(c1.tokens) if t.index == token.index), -1)
            index2 = next((i for i, t in enumerate(c2.tokens) if t.index == token.index), -1)

            if index1 > -1 and index2 > -1 and not token.nonce:
                c1_alpha = self._alpha(token, tokens, c1)
                c2_alpha = self._alpha(token, tokens, c2)

                c1.tokens[index1].weight = (c1.tokens[index1].weight * c1_alpha) + (c1_alpha / (len(self.categories)+1))
                c2.tokens[index2].weight = (c2.tokens[index2].weight * c2_alpha) + (c2_alpha / (len(self.categories)+1))

        ntokens = self._non_intersect(c1.tokens, c2.tokens)

        for token in ntokens:
            index1 = next((i for i, t in enumerate(c1.tokens) if t.index == token.index), -1)
            index2 = next((i for i, t in enumerate(c1.tokens) if t.index == token.index), -1)

            if index1 > -1:
                c1_alpha = self._alpha(token, tokens, c1)
                c1.tokens[index1].weight += (c1_alpha / (len(self.categories)+1))
            elif index2 > -1:
                c2_alpha = self._alpha(token, tokens, c2)
                c2.tokens[index2].weight += (c2_alpha / (len(self.categories)+1))

        return c1, c2


    def predict(self, text, max_results=2):
        tokens, _ = Tokenize(self.word_pooling).apply(text, self.vocab)  # Assuming tokenize returns a tuple
        results = []

        for token in tokens:
            ts = [t for t in self.tensors if t.word_id == token.index]
            if ts:
                for t in ts:
                    category = next((c.name for c in self.categories if c.id == t.category_id), None)
                    if category:
                        index = next((i for i, r in enumerate(results) if r['category'] == category), -1)
                        if index > -1:
                            results[index]['category_count'] += 1
                            results[index]['weight'] += (t.weight > self.dropout and t.weight or 0) + ( results[index]['category_count'] + len(tokens))   # Simplified conditional
                        else:
                            results.append({'category_id': t.category_id, 'category': category, 'weight': (t.weight > self.dropout and t.weight or 0), 'category_count': 1, 'input_token_count': len(tokens)})

        results = self._softmax1(results)
        results.sort(key=lambda r: r['confidence'], reverse=True)
        return self._limiter(results, max_results)


    def save_model(self, filename):
        print(f"SaveModel: {filename} Tensors: {len(self.tensors)}")

        model = Model(tensors = self.tensors, vocab = self.vocab, categories = self._fit_categories_small(self.categories), vectors = self._fit_vectors_to_dict(self.vectors))

        print(f"Tensors: {len(self.tensors)}")
        print(f"Vocab: {len(self.vocab)}")
        print(f"Categories: {len(self.categories)}")
        print(f"Vectors: {len(self.vectors)}")

        # Save as pickle (binary serialization, a replacement for protobuf)
        # with open(filename, "wb") as f:
        #    pickle.dump(model, f)
        ProtobufConverter.save_to_file(model, filename)






    def save_json_model(self, filename):
        options = {"indent": 4}  # For readable JSON

        model = Model(tensors = self.tensors, vocab = self.vocab, categories = self._fit_categories_small(self.categories), vectors = self._fit_vectors_to_dict(self.vectors))

        with open(f"{filename}.json", "w") as f:
            json.dump(model, f, **options)


    def load_model(self, filename):
        #with open(filename, "rb") as f:  # Load from pickle file
        #    model = pickle.load(f)
        model = ProtobufConverter.load_from_file(filename)

        print(f"Tensors: {len(model.tensors)}")
        print(f"Vocab: {len(model.vocab)}")
        print(f"Categories: {len(model.categories)}")

        return (model.tensors, model.vocab, model.categories,
                self._fit_vectors_from_dict(model.vectors)) # Convert from dictionary


    def _fit_vectors_to_dict(self, vectors):  # Helper function
        word_vector = {"embeddings": []}
        for vector in vectors:
            embedding = {"values": vector} # Directly assign the list
            word_vector["embeddings"].append(embedding)
        return word_vector


    def _fit_vectors_from_dict(self, vectors_dict):  # Helper function
        vectors = []
        for embedding in vectors_dict.embeddings:
            vectors.append(embedding.values)
        return vectors


    def _fit_categories_small(self, categories):  # Helper function
        return [{"id": cat.id, "name": cat.name} for cat in categories]


    def _fit_categories_from_small(self, categories_small) -> List[Category]:  # Helper function
        return [Category(id=cat.id, name=cat.name) for cat in categories_small]


    def _intersect(self, arr1, arr2):
        result = []
        for token1 in arr1:
            for token2 in arr2:
                if token1.index == token2.index and not token1.nonce:
                    result.append(token1)
        return result

    def _non_intersect(self, arr1, arr2):
        indices_lista2 = {t.index for t in arr2}
        result = [t for t in arr1 if t.index not in indices_lista2] + [t for t in arr2 if not any(t1.index == t.index for t1 in arr1)]
        return result

    def _alpha(self, selected_token, intersect_tokens, cat):
        _alpha = 0.0
        position = selected_token.position

        for token in intersect_tokens:
            if token.index != selected_token.index:
                index_cat = next((i for i, t in enumerate(cat.tokens) if t.index == token.index), -1)
                if index_cat == -1:
                    continue

                near_position = self._alpha_minor_distance_position(position, cat.tokens[index_cat].position)
                near_distance = abs(near_position - index_cat)
                if near_distance == 0:
                    near_distance = 10000
                _alpha += 1.0 / (near_distance ** self.distance_temperature)
        return _alpha / (len(intersect_tokens) + 1)


    def _alpha_minor_distance_position(self, position, ids):
        dist = 10000
        for pos in position:
            for id in ids:
                dst = abs(pos - id)
                if dst < dist:
                    dist = dst
        return dist

    def _calc_ratio(self, c1, c2, t_count, v_count):
        r = 2
        if c1 > c2:
            return (r * (c1 / c2)) / math.sqrt(t_count / v_count)
        else:
            return (r * (c1 / c2)) / math.sqrt(t_count / v_count)

    def _softmax1(self, results):
        sum_exp = sum(math.exp(result['weight']) for result in results)
        for result in results:
            result['exponential'] = math.exp(result['weight'])
            result['confidence'] = result['exponential'] / sum_exp
        return results


    def _softmax(self, results):
        total_weight = sum(result['weight'] for result in results)
        for result in results:
            result['confidence'] = result['weight'] / total_weight
        return results

    def _limiter(self, results, max_results):
        if len(results) > max_results:
            del results[max_results:]  # Efficiently remove excess elements
        return results

    def export(self):
        return self.categories, self.tensors, self.vocab

    def _intersect_count(self, t1, t2): # Renamed for clarity
        count = 0
        for word1 in t1:
            for word2 in t2:  # Corrected loop to iterate over t2
                if word1 == word2:
                    count += 1
        return count

    def _similarity(self, t1, t2):
        t1_arr = self._sanitize(t1).split()  # Split on whitespace by default
        t2_arr = self._sanitize(t2).split()

        intersect = self._intersect_count(t1_arr, t2_arr)
        return (2.0 * intersect) / (len(t1_arr) + len(t2_arr))

    def _sanitize(self, text):
        text = text.lower()
        for char in ",.*:'?!":  # More efficient replacement
            text = text.replace(char, "")
        return text.replace("  ", " ")  # Keep this replacement

    def none_category(self, c, a, b, w):
        r = c / (a + b)  # Simplified division
        return r > c * self.non_category_threshold

    def print_results(self, results):  # More descriptive name
        for result in results:
            t = sum(1 for tensor in self.tensors if tensor.category_id == result['category_id'])
            print(f"{result['category_id']} {result['category']} {result['confidence']} {t} {result['category_count']} {result['input_token_count']} {result['weight']} {self.none_category(t, result['category_count'], result['input_token_count'], result['weight'])}")

    def print_embeddings(self, text):
        tokens = Tokenize.instance(self.word_pooling).apply(text, self.vocab)[0]

        for token in tokens:
            tensor = max((t for t in self.tensors if t.word_id == token.index), key=lambda t: t.weight, default=None)

            if tensor:
                category_name = next((c.name for c in self.categories if c.id == tensor.category_id), "None Category")
                print(f"{self.vocab[token.index].word} {category_name} {tensor.weight}")
            else:
                print(f"{self.vocab[token.index].word} None Category")


    def print_vocab(self, vocab_list=None): # Added optional parameter
        vocab_list = vocab_list or self.vocab # Use self.vocab if no list is provided
        for vocab_item in vocab_list:
            print(f"{vocab_item.word} {vocab_item.word_partial}")

    def print_tensors(self, tensor_list=None): # Added optional parameter
        tensor_list = tensor_list or self.tensors # Use self.tensors if no list is provided
        for tensor in tensor_list:
            print(f"{tensor.word_id} {tensor.category_id}")

    def print_category_tokens(self, category):
        print(f"{category.name} -----------------------------------------------------")
        for i, token in enumerate(category.tokens):
            print(f"{i} {self.vocab[token.index].word} {token.count}")

    def print_vectors(self, vectors_list):
        for vector in vectors_list:
            print(", ".join(map(str, vector)))  # More concise printing

    def print_categories(self):
        for category in self.categories:
            print(f"{category.id} {category.name}")

In [ ]:
tokz = Tokenize(word_pooling=1)


def Training():
    list1 = [
        "a shigelose é uma infecção intestinal causada por bactérias do gênero shigella. a espécie shigella dysenteriae é considerada a mais grave e é capaz de causar diarreia com sangue e mucosas, além de febre, dor abdominal e náuseas. a transmissão pode ocorrer por água ou alimentos contaminados, além do contato direto com fezes infectadas.",
        "a febre paratifóide b é uma infecção bacteriana aguda causada pela bactéria salmonella paratyphi b.",
        "essa categoria inclui outras intoxicações alimentares bacterianas não especificadas, exceto aquelas causadas por salmonella, shigella, campylobacter e clostridium perfringens",
        "doença parasitária causada pelo protozoário giardia lamblia, que afeta principalmente o trato gastrointestinal humano.",
        "doença infecciosa causada pela bactéria mycobacterium tuberculosis, que afeta principalmente os pulmões e pode ser transmitida pelo ar."
    ]

    list2 = [
        "doenças causadas pela exposição a poluentes presentes no solo, como metais pesados e substâncias químicas tóxicas.",
        "a exposição ocupacional a agentes tóxicos em outras indústrias pode ocorrer em diversos setores, como em trabalhadores que lidam com produtos químicos, metais pesados, solventes e outras substâncias tóxicas. essa exposição pode causar problemas neurológicos, respiratórios, dermatológicos e até mesmo câncer.",
        "essa categoria inclui doenças e transtornos relacionados ao trabalho que não se enquadram em outras categorias específicas. alguns exemplos incluem dores musculares e articulares, fadiga, ansiedade e estresse ocupacional.",
        "problema relacionado com o medo ou possibilidade de perder o emprego atual, o que pode levar a ansiedade, estresse e outros problemas emocionais",
        "esta categoria inclui problemas relacionados com a educação e a alfabetização que não se enquadram nas categorias anteriores, como dificuldades de aprendizagem, transtornos de déficit de atenção e hiperatividade, entre outros.",
        "analfabetismo é a incapacidade de ler e escrever, enquanto baixa escolaridade se refere a indivíduos que possuem pouca escolaridade formal. ambas são consideradas fatores de risco para diversas doenças e problemas de saúde, como a falta de acesso à informação e dificuldade em compreender informações importantes sobre saúde."
    ]

    list3 = [
        "não foi encontrado dados precisos sobre a taxa de ocorrência e letalidade específicas para essa doença, mas pode-se considerar os dados gerais da doença diverticular do intestino grosso",
        "a fissura anal aguda é uma lesão na pele que reveste o ânus e pode ser bastante dolorosa. ela ocorre principalmente em adultos jovens e saudáveis e é mais comum em mulheres do que em homens.",
        "o volvo é uma condição rara e grave que ocorre quando uma porção do intestino se torce em torno de sua própria base, interrompendo o fluxo sanguíneo para essa área. isso pode levar à morte do tecido intestinal e à perfuração do intestino, o que pode levar a uma infecção grave.",
        "a fístula anorretal é uma comunicação anormal que se forma entre o canal anal ou reto e a pele ao redor do ânus. a causa mais comum é uma infecção do trato anal ou reto, que pode levar a um abscesso que não foi tratado adequadamente. ",
        "o prolapso anal é uma condição em que a mucosa retal se projeta através do ânus, podendo ser classificado em três tipos: prolapso mucoso, prolapso retal completo e prolapso retal interno.",
        "a proctite por radiação é uma inflamação da mucosa retal que ocorre como resultado da radioterapia pélvica. é comum em pacientes com câncer de próstata, câncer de reto ou outros tipos de câncer pélvico que são tratados com radioterapia."
    ]



    vocab = Tokenize.instance(word_pooling=1).vocab([list1, list2, list3])
    #Tokenize.instance().print_vocab(vocab)

    clasf = Classify.instance()
    clasf.vocab(vocab)
    #clasf.word_pooling(1)
    clasf.model("word-conquerer-attention-model.dat")
    clasf.add_category("CatA", list1)
    clasf.add_category("CatZ", list2)
    clasf.add_category("CatK", list3)
    clasf.train()


Training()

Training Start
Training CatZ -> CatK 4/3 134% ##################################
SaveModel: word-conquerer-attention-model.dat Tensors: 333
Tensors: 333
Vocab: 272
Categories: 3
Vectors: 17
Training Finished


In [ ]:
clasf = Classify.instance().model("word-conquerer-attention-model.dat", True)
r1 = clasf.predict("problema relacionado com o medo ou possibilidade de perder o emprego atual ansiedade, que afeta principalmente", 3)
clasf.print_results(r1)
print()

#clasf.print_embeddings("problema relacionado com o medo ou possibilidade de perder o emprego atual ansiedade, que afeta principalmente")
#print()

Tensors: 333
Vocab: 272
Categories: 3
2 CatZ 1.0 119 15 17 365.461753803121 False
1 CatA 1.2150854493257377e-80 88 9 17 181.44976076658006 False
3 CatK 2.0075382750554658e-92 126 8 17 154.32083448035593 False



In [ ]:
class Generative:
    def __init__(self, categories: None, tensors:  None, vocab: None):
        self.word_pooling = 1.0
        self.categories = categories if categories else []
        self.tensors = tensors if tensors else []
        self.vocab = vocab if vocab else []
        self.vectors = []
        self.model_path = ""


    @staticmethod
    def instance(categories = None, tensors = None, vocab =  None):
        return Generative(categories, tensors, vocab)

    def word_pooling(self, word_pooling: float):
        self.word_pooling = word_pooling
        return self

    def model(self, model_path: str, read: bool = False):
        print(f"Model: {model_path} {os.path.exists(model_path)} {read}")
        if read and os.path.exists(model_path):
            self.tensors, self.vocab, _categories, self.vectors = self.load_model(model_path)
            self.categories = self._fit_categories_from_small(_categories)
        return self

    def next_word(self, word: str, useContext: bool = False):
        word_id = next((i for i, v in enumerate(self.vocab) if v.word == word), -1)
        _vectors = self.vectors

        if useContext:
            classifier = Classify.instance("", self.vocab, self.categories, self.tensors, self.vectors)
            results = classifier.predict(word, 1)
            t = sum(1 for c in self.tensors if c.category_id == results[0]["category_id"])

            if not classifier.none_category(t, results[0]["category_count"], results[0]["input_token_count"], results[0]["weight"]):
                _vectors = [v for v in self.vectors if v[0] == results[0]["category_id"]]

        for lst in _vectors:
            index = lst.index(word_id) if word_id in lst else -1
            if index > -1:
                for i in range(index, len(lst) - index):
                    if self.vocab[lst[i]].word == "<end>":
                        print()
                        break
                    print(f"{self.vocab[lst[i]].word} ", end="")


    def qna(self, phrase: str, useContext: bool = False):
        print(f"{self.capitalizar_primeira_letra(phrase)} ", end="")
        _vectors = self.vectors

        if useContext:
            classifier = Classify.instance("",self.vocab, self.categories, self.tensors, self.vectors)
            results = classifier.predict(phrase, 1)
            classifier.print_results(results)
            t = sum(1 for c in self.tensors if c.category_id == results[0]["category_id"])
            if not classifier.none_category(t, results[0]["category_count"], results[0]["input_token_count"], results[0]["weight"]) or results[0]["confidence"] > 0.5:
                _vectors = [v for v in self.vectors if v[0] == results[0]["category_id"]]

        _vec_db = []
        words = self.pontuation(phrase).lower().split()
        words_ids = [next((i for i, v in enumerate(self.vocab) if v.word == word), -1) for word in words]



        for word_id in words_ids:
            at = [t for t in self.tensors if t.word_id == word_id]
            at.sort(key=lambda t: t.weight, reverse=True)
            w = at[0].weight if at else 0

            if w > 0:
                for i, vec in enumerate(_vectors):
                    index = vec.index(word_id) if word_id in vec else -1
                    if index > -1:
                        v_index = next((j for j, v in enumerate(_vec_db) if v.index == i), -1)
                        if v_index > -1:
                            _vec_db[v_index].count += 1
                            _vec_db[v_index].weight += self.beta_distance(vec, words_ids, word_id)
                        else:
                            _vec_db.append(GenVec(index=i, count=1, weight=w))

        _vec_db.sort(key=lambda v: v.weight, reverse=True)
        nextCaption = True

        for i in range(1, len(_vectors[_vec_db[0].index])):
            word = self.vocab[_vectors[_vec_db[0].index][i]].word
            if nextCaption and word != "<end>":
                word = word.title()

            if word == ".":
                nextCaption = True
            else:
                nextCaption = False

            if word == "<end>":
                break
            print(f"{word} ", end="", flush=True)
            time.sleep(0.15)

        print()


    def analyze(self, phrase: str, useContext: bool = False):
        print(f"{self.capitalizar_primeira_letra(phrase)} ", end="")
        _vectors = self.vectors

        if useContext:
            classifier = Classify.instance(self.categories, self.tensors, self.vocab, self.vectors)
            results = classifier.predict(phrase, 1)
            t = sum(1 for c in self.tensors if c.category_id == results[0].category_id)

            if not classifier.none_category(t, results[0].category_count, results[0].input_token_count, results[0].weight) or results[0].weight > 0.5:
                _vectors = [v for v in self.vectors if v[0] == results[0].category_id]

        words = self.pontuation(phrase).lower().split()
        for word in words:
            print(f"{word} ")
            word_id = next((i for i, v in enumerate(self.vocab) if v.word == word), -1)
            _tensors = [t for t in self.tensors if t.word_id == word_id]
            _tensors.sort(key=lambda t: t.weight, reverse=True)

            for t in _tensors:
                category = next(c for c in self.categories if c.id == t.category_id)
                print(f"\t {category.name} {t.weight}")


    def beta_distance(self, lst: List[int], words_ids: List[int], word_id: int) -> float:
        d = 1000000
        p_word = lst.index(word_id) if word_id in lst else -1

        if p_word > -1:
            for id in words_ids:
                if id != word_id:
                    p_id = lst.index(id) if id in lst else -1
                    if p_id > -1:
                        dist = abs(p_id - p_word)
                        if dist < d:
                            d = dist

        return 1 / d



    def load_model(self, filename):
        #with open(filename, "rb") as f:  # Load from pickle file
        #    model = pickle.load(f)
        model = ProtobufConverter.load_from_file(filename)

        print(f"Tensors: {len(model.tensors)}")
        print(f"Vocab: {len(model.vocab)}")
        print(f"Categories: {len(model.categories)}")

        return (model.tensors, model.vocab, model.categories,
                self._fit_vectors_from_dict(model.vectors)) # Convert from dictionary


    def _fit_vectors(self, vectors: 'WordVector') -> List[List[int]]:
        return [list(emb.values) for emb in vectors.embeddings]

    def _fit_vectors_from_dict(self, vectors_dict):  # Helper function
        vectors = []
        for embedding in vectors_dict.embeddings:
            vectors.append(embedding.values)
        return vectors

    def _fit_categories_from_small(self, categories_small) -> List[Category]:  # Helper function
        return [Category(id=cat.id, name=cat.name) for cat in categories_small]


    def vocab(self) -> List['Vocab']:
        return self.vocab

    @staticmethod
    def word_pooling(word: str, rate: float) -> str:
        length = len(word)
        if length <= 4:
            return word
        pad = int(np.ceil(length * rate))
        return word[:pad]

    def pontuation(self, text: str) -> str:
        text = text.lower().replace(",", "").replace(".", "").replace("*", "").replace(":", "").replace("'", "").replace("?", "").replace("!", "").replace("  ", " ")
        return text

    def pontuation_r(self, text: str) -> str:
        text = text.lower().replace(" , ", ",").replace(" . ", ".").replace(" * ", "*").replace(" : ", ":").replace(" ' ", "'")
        return text

    @staticmethod
    def print_vectors(self, lst: List[List[int]]):
        for vector in lst:
            print(", ".join(map(str, vector)))

    def print_gen_vec(self, lst: List['GenVec']):
        for genVec in lst:
            print(f"{genVec.index} {genVec.count} {genVec.weight} {genVec.count * genVec.weight}")

    def print_phrase(self, lst: List[int]):
        for i in range(1, len(lst)):
            print(f"{self.vocab[lst[i]].word} ", end="")
        print("\n----------------------------------------------------------------")

    @staticmethod
    def capitalizar_primeira_letra(input: str) -> str:
        if not input:
            return input
        return input[0].upper() + input[1:]

In [ ]:
genai = Generative.instance().model("cid10-1wp-model-2025-05-23.dat", True)
genai.qna("a neoplasia maligna da glote é um tipo de câncer", True)

Model: cid10-1wp-model-2025-05-23.dat True True
Tensors: 35897
Vocab: 11763
Categories: 22
A neoplasia maligna da glote é um tipo de câncer 3 C 0.9999999997048128 1656 11 11 176.0727178293969 False
A neoplasia maligna da glote é um tipo de câncer que afeta as pregas vocais na laringe , sendo mais comum em homens acima de 50 anos . Os principais fatores de risco incluem tabagismo e consumo excessivo de álcool . 


In [ ]:
class Seq2Seq:
    def __init__(self, model_path = None, vocab = None, categories = None, tensors = None, vectors = None):
        self.word_pooling = 1.0
        self.distance_temperature = 1.0
        self.model_path = model_path if model_path else ""
        if vocab != None: self.vocab = vocab or []
        self.categories = categories if categories else []
        self.tensors = tensors if tensors else []
        self.vectors = vectors if vectors else []

    @staticmethod
    def instance(model_path = None, vocab = None, categories = None, tensors = None, vectors = None):
        return Seq2Seq(model_path, vocab, categories, tensors, vectors)

    def word_pooling(self, word_pooling: float):
        self.word_pooling = word_pooling
        return self

    def vocab(self, vocab: List['Vocab']):
        self.vocab = vocab
        return self

    def distance_temperature(self, distance_temperature):
        self.distance_temperature = distance_temperature
        return self

    def model(self, model_path: str, read: bool = False):
        self.model_path = model_path
        print(f"Model: {model_path} {os.path.exists(model_path)} {read}")
        if read and os.path.exists(model_path):
            self.tensors, self.vocab, _categories, self.vectors = self.load_model(model_path)
            self.categories = self._fit_categories_from_small(_categories)
        return self

    def add_category(self, name: str, question: str, answer: str):
        index = next((i for i, c in enumerate(self.categories) if c.name == name), -1)

        if index == -1:
            id = len(self.categories) + 1
            t = Category(id=id, name=name)
            t.add_tokens(Tokenize.instance(self.word_pooling).apply(question, self.vocab))
            self.categories.append(t)
            ids = Tokenize.instance(self.word_pooling).embedding(id, answer, self.vocab)
            self.vectors.append(ids)
        else:
            self.categories[index].add_tokens(Tokenize.instance(self.word_pooling).apply(question, self.vocab))
            ids = Tokenize.instance(self.word_pooling).embedding(self.categories[index].id, answer, self.vocab)
            self.vectors.append(ids)
        return self

    def train(self):
        print("Training Start")
        self.race()
        self.fit()

        if not self.model_path:
            self.model_path = f"conquerer-{self.word_pooling}-model.bin"
        self.save_model(self.model_path)
        print("Training Finished")

    def race(self):
        percent = 0
        all = sum(range(1, len(self.categories)))

        for i in range(len(self.categories) - 1):
            for j in range(i + 1, len(self.categories)):
                c1, c2 = self.conquerer(self.categories[i], self.categories[j])
                self.categories[i] = c1
                self.categories[j] = c2
                percent += 1
                p = (100 * (percent + 1) / all)
                print(f"Training {c1.name} -> {c2.name} {percent + 1}/{all} {math.ceil(p)}% ", end="")
                print("#" * int(math.ceil(p / 4)), end="")
                print()

    def fit(self):
        self.tensors.clear()
        for category in self.categories:
            for token in category.tokens:
                self.tensors.append(Tensor(category_id=category.id, word_id=token.index, weight=token.weight))

    def conquerer(self, c1: Category, c2: Category) -> Tuple[Category, Category]:
        tokens = self.intersect(c1.tokens, c2.tokens)
        for token in tokens:
            index1 = next((i for i, t in enumerate(c1.tokens) if t.index == token.index), -1)
            index2 = next((i for i, t in enumerate(c2.tokens) if t.index == token.index), -1)
            if index1 > -1 and index2 > -1 and not token.nonce:
                c1Alpha = self.alpha(token, tokens, c1)
                c2Alpha = self.alpha(token, tokens, c2)

                c1.tokens[index1].weight = (c1.tokens[index1].weight * c1Alpha) + (c1Alpha / (len(self.categories)+1))
                c2.tokens[index2].weight = (c2.tokens[index2].weight * c2Alpha) + (c2Alpha / (len(self.categories)+1))
        ntokens = self.non_intersect(c1.tokens, c2.tokens)
        for token in ntokens:
            index1 = next((i for i, t in enumerate(c1.tokens) if t.index == token.index), -1)
            index2 = next((i for i, t in enumerate(c2.tokens) if t.index == token.index), -1)
            if index1 > -1:
                c1Alpha = self.alpha(token, tokens, c1)
                c1.tokens[index1].weight += (c1Alpha / (len(self.categories)+1))
            elif index2 > -1:
                c2Alpha = self.alpha(token, tokens, c2)
                c2.tokens[index2].weight += (c2Alpha / (len(self.categories)+1))
        return c1, c2

    def predict(self, phrase: str, threshold: float = 0.4):
        print(f"{self.capitalizar_primeira_letra(phrase)} ", end="")
        print()
        _vectors = self.vectors

        classifier = Classify.instance("", self.vocab,self.categories, self.tensors, self.vectors)
        results = [r for r in classifier.predict(phrase, 1) if r["confidence"] > threshold]

        if(len(results) == 0):
            print("Não fui treinado com esse conhecimento!")
            return

        index = int(results[0]["category"])

        nextCaption = True
        for i in range(1, len(_vectors[index])):
            word = self.vocab[_vectors[index][i]].word
            if nextCaption and word != "<end>":
                word = word.title()
            if word == ".":
                nextCaption = True
            else:
                nextCaption = False
            if word == "<end>":
                break
            print(f"{word} ", end="", flush=True)
            time.sleep(0.15)
        print()


    def save_model(self, filename):
        print(f"SaveModel: {filename} Tensors: {len(self.tensors)}")

        model = Model(tensors = self.tensors, vocab = self.vocab, categories = self._fit_categories_small(self.categories), vectors = self._fit_vectors_to_dict(self.vectors))

        print(f"Tensors: {len(self.tensors)}")
        print(f"Vocab: {len(self.vocab)}")
        print(f"Categories: {len(self.categories)}")
        print(f"Vectors: {len(self.vectors)}")

        # Save as pickle (binary serialization, a replacement for protobuf)
        # with open(filename, "wb") as f:
        #    pickle.dump(model, f)
        ProtobufConverter.save_to_file(model, filename)


    def save_json_model(self, filename):
        options = {"indent": 4}  # For readable JSON

        model = {
            "tensors": self.tensors,
            "vocab": self.vocab,
            "categories": self._fit_categories_small(self.categories),
            "vectors": self._fit_vectors_to_dict(self.vectors)
        }

        with open(f"{filename}.json", "w") as f:
            json.dump(model, f, **options)


    def load_model(self, filename):
        #with open(filename, "rb") as f:  # Load from pickle file
        #    model = pickle.load(f)
        model = ProtobufConverter.load_from_file(filename)

        print(f"Tensors: {len(model.tensors)}")
        print(f"Vocab: {len(model.vocab)}")
        print(f"Categories: {len(model.categories)}")

        return (model.tensors, model.vocab, model.categories,
                self._fit_vectors_from_dict(model.vectors)) # Convert from dictionary


    def _fit_vectors_to_dict(self, vectors):  # Helper function
        word_vector = {"embeddings": []}
        for vector in vectors:
            embedding = {"values": vector} # Directly assign the list
            word_vector["embeddings"].append(embedding)
        return word_vector


    def _fit_vectors_from_dict(self, vectors_dict):  # Helper function
        vectors = []
        for embedding in vectors_dict.embeddings:
            vectors.append(embedding.values)
        return vectors

    def _fit_categories_small(self, categories):  # Helper function
        return [{"id": cat.id, "name": cat.name} for cat in categories]


    def _fit_categories_from_small(self, categories_small) -> List[Category]:  # Helper function
        return [Category(id=cat.id, name=cat.name) for cat in categories_small]

    def intersect(self, arr1: List['Token'], arr2: List['Token']) -> List['Token']:
        return [t1 for t1 in arr1 for t2 in arr2 if t1.index == t2.index and not t1.nonce]

    def non_intersect(self, arr1: List['Token'], arr2: List['Token']) -> List['Token']:
        indices_lista2 = {t.index for t in arr2}
        return [t for t in arr1 if t.index not in indices_lista2] + [t for t in arr2 if not any(t1.index == t.index for t1 in arr1)]

    def alpha(self, selected_token: 'Token', intersect_tokens: List['Token'], cat: 'Category') -> float:
        _alpha = 0.0
        position = selected_token.position
        for token in intersect_tokens:
            if token.index != selected_token.index:
                index_cat = next((i for i, t in enumerate(cat.tokens) if t.index == token.index), -1)
                if index_cat == -1:
                    continue
                near_position = self.alpha_minor_distance_position(position, cat.tokens[index_cat].position)
                near_distance = abs(near_position - index_cat)
                if near_distance == 0:
                    near_distance = 10000
                _alpha += 1.0 / (near_distance ** self.distance_temperature)
        return _alpha / (len(intersect_tokens) + 1)

    def alpha_minor_distance_position(self, position: List[int], ids: List[int]) -> int:
        dist = 10000
        rid = 0
        for pos in position:
            for id in ids:
                dst = abs(pos - id)
                if dst < dist:
                    dist = dst
                    rid = id
        return dist

    def pontuation(self, text: str) -> str:
        text = text.lower().replace(",", "").replace(".", "").replace("*", "").replace(":", "").replace("'", "").replace("?", "").replace("!", "").replace("  ", " ")
        return text

    @staticmethod
    def capitalizar_primeira_letra(input: str) -> str:
        if not input:
            return input
        return input[0].upper() + input[1:]

In [ ]:
list4 = [
    ["Qual é o maior planeta do sistema solar?", "Júpiter"],
    ["Quem pintou a Mona Lisa?", "Leonardo da Vinci"],
    ["Qual é a capital do Japão?", "Tóquio"],
    ["Quem escreveu 'Dom Quixote'?", "Miguel de Cervantes"],
    ["Quantos ossos tem o corpo humano adulto?", "206"],
    ["Qual é o elemento químico representado pelo símbolo 'O'?", "Oxigênio"],
    ["Quem foi o primeiro homem a pisar na Lua?", "Neil Armstrong"],
    ["Qual é a moeda oficial do Reino Unido?", "Libra esterlina"],
    ["Qual é o maior oceano do mundo?", "Oceano Pacífico"],
    ["Quantos lados tem um hexágono?", "Seis"],
    ["Quem descobriu o Brasil?", "Pedro Álvares Cabral"],
    ["Qual é a montanha mais alta do mundo?", "Monte Everest"],
    ["Em que ano começou a Primeira Guerra Mundial?", "1914"],
    ["Qual é a capital da França?", "Paris"],
    ["Quem desenvolveu a Teoria da Relatividade?", "Albert Einstein"],
    ["Qual é o metal líquido à temperatura ambiente?", "Mercúrio"],
    ["Quantos estados tem o Brasil?", "26 estados e um Distrito Federal"],
    ["Quem foi o primeiro presidente do Brasil?", "Deodoro da Fonseca"],
    ["Qual é o animal terrestre mais rápido?", "Guepardo"],
    ["Quem é o autor de 'Romeu e Julieta'?", "William Shakespeare"],
    ["Qual é a fórmula química da água?", "H₂O"],
    ["Quem inventou a lâmpada elétrica?", "Thomas Edison"],
    ["Qual é o maior deserto do mundo?", "Deserto da Antártida"],
    ["Qual país é famoso pela Torre Eiffel?", "França"],
    ["Quem criou a teoria da evolução das espécies?", "Charles Darwin"],
    ["Quantos planetas existem no sistema solar?", "Oito"],
    ["Qual é o nome do maior rio do mundo em volume de água?", "Rio Amazonas"],
    ["Qual é o nome da maior floresta tropical do mundo?", "Amazônia"],
    ["Quem escreveu 'O Pequeno Príncipe'?", "Antoine de Saint-Exupéry"],
    ["Em que ano terminou a Segunda Guerra Mundial?", "1945"],
    ["Qual é o órgão responsável por bombear sangue no corpo humano?", "Coração"],
    ["Qual é a capital da Austrália?", "Camberra"],
    ["Quem foi o líder da Revolução Russa de 1917?", "Vladimir Lênin"],
    ["Qual é o nome do gás essencial para a respiração humana?", "Oxigênio"],
    ["Quem pintou 'A Última Ceia'?", "Leonardo da Vinci"],
    ["Qual é o país com a maior população do mundo?", "China"],
    ["Quem escreveu 'A Divina Comédia'?", "Dante Alighieri"],
    ["Qual é o nome do processo em que as plantas produzem seu próprio alimento?", "Fotossíntese"],
    ["Qual é o maior mamífero do mundo?", "Baleia-azul"],
    ["Quem foi o fundador do Império Mongol?", "Genghis Khan"]
]



vocab = Tokenize.instance(word_pooling=1).vocab(list4)

model = Seq2Seq.instance()
model.vocab(vocab)
model.model("seq2seq-model.dat")


for i in range(len(list4)):
   model.add_category(str(i), list4[i][0], list4[i][1])

model.train()

In [ ]:
model = Seq2Seq.instance().model("seq2seq-model.dat", True)
model.predict("Qual é o maior mamífero do mundo?", 0.0001)

Model: seq2seq-model.dat True True
Tensors: 340
Vocab: 193
Categories: 40
Qual é o maior mamífero do mundo? 
Baleia-Azul 
